In [2]:
import xarray as xr
import numpy as np

import basal_melt_NEMO.metrics_functions as mf

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [3]:
inputpath_cas_closed='/data/cdelaver/n42tm21/'
inputpath_ethe_v4='/thredds/tgcc/store/p48ethe/NEMO/DEVT/clim/eOR1Si3Pv423-CORE-clim-02/OCE/Analyse/TS_MO/'
inputpath_ethe_v5='/thredds/tgcc/store/p48ethe/NEMO/DEVT/clim/eOR1Si3Pv5rc-CORE-clim-RK3-02/OCE/Analyse/TS_MO/'

In [ ]:
Cas_year_100 = xr.open_mfdataset(inputpath_cas_closed + 'n42tm21_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_T.nc')
Cas_year_1000 = 

for yy in range(9):
    yy1=yy+1
    file_open = xr.open_mfdataset(inputpath_open + 'n42openc_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_T.nc')
    file_closed = xr.open_mfdataset(inputpath_closed + 'n42tm21_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_T.nc')
    file_open_list.append(file_open)
    file_closed_list.append(file_closed)
file_T_open_all = xr.concat(file_open_list, dim='time_counter')
file_T_closed_all = xr.concat(file_closed_list, dim='time_counter')

In [4]:
def weighted_temporal_mean(ds, var):
    """
    weight by days in each month
    """
    # Determine the month length
    month_length = ds.time.dt.days_in_month

    # Calculate the weights
    wgts = month_length.groupby("time.year") / month_length.groupby("time.year").sum()

    # Make sure the weights in each year add up to 1
    np.testing.assert_allclose(wgts.groupby("time.year").sum(xr.ALL_DIMS), 1.0)

    # Subset our dataset for our variable
    obs = ds[var]

    # Setup our masking for nan values
    cond = obs.isnull()
    ones = xr.where(cond, 0.0, 1.0)

    # Calculate the numerator
    obs_sum = (obs * wgts).resample(time="AS").sum(dim="time")

    # Calculate the denominator
    ones_out = (ones * wgts).resample(time="AS").sum(dim="time")

    # Return the weighted average
    return obs_sum / ones_out

In [9]:
ethe_v4_thetao = xr.open_dataset(inputpath_ethe_v4 + 'eOR1Si3Pv423-CORE-clim-02_19480101_20471231_1M_thetao.nc').rename({'time_counter': 'time'})
ethe_v5_thetao = xr.open_dataset(inputpath_ethe_v5 + 'eOR1Si3Pv5rc-CORE-clim-RK3-02_19480101_20471231_1M_thetao.nc').rename({'time_counter': 'time'})

In [10]:
ethe_v4_thetao

<xarray.Dataset>
Dimensions:               (y: 331, x: 360, deptht: 75, time: 1200,
                           axis_nbounds: 2)
Coordinates:
    nav_lat               (y, x) float32 ...
    nav_lon               (y, x) float32 ...
  * deptht                (deptht) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    time_centered         (time) object ...
  * time                  (time) object 1948-01-16 12:00:00 ... 2047-12-16 12...
Dimensions without coordinates: y, x, axis_nbounds
Data variables:
    time_centered_bounds  (time, axis_nbounds) object ...
    thetao                (time, deptht, y, x) float32 ...
Attributes:
    name:         eOR1Si3Pv423-CORE-clim-02_1m_grid_T
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Aug-30 11:53:50 GMT
    uuid:         d12f3603-e365-44de-8b47-e760c2804922
    LongName:     ORCA1_SI3_PISCES NEMO configuration
    history:      Sat Aug 31 02:20:06 2024: ncrcat -C --buffer_size 838860800...
    NCO:          netCDF Operators version 5.0.1 (Homepage = http://nco.sf.ne...

In [ ]:
weighted_temporal_mean(ethe_v4_thetao, 'thetao')

In [ ]:
print('OK done')

In [ ]:
ethe_v4_so = xr.open_dataset(inputpath_ethe_v4 + 'eOR1Si3Pv423-CORE-clim-02_19480101_20471231_1M_so.nc')
ethe_v5_so = xr.open_dataset(inputpath_ethe_v5 + 'eOR1Si3Pv5rc-CORE-clim-RK3-02_19480101_20471231_1M_so.nc')